<a id="Bi-LSTM CRF"></a>

# Bi-LSTM CRF Model

We will leverage a Bi-LSTM CRF model as the baseline tagger.

This notebook explores the construction of the tagger.

In [2]:
from pathlib import Path
import pandas as pd

In [3]:
ROOT_DIR = Path('notebooks/eda.ipynb').resolve().parents[2]
DATA_DIR = ROOT_DIR / "data"
PREPARED_DIR = DATA_DIR / "prepared"

In [4]:
from ast import literal_eval


df = pd.read_csv(PREPARED_DIR / "master.csv")
df["tags"] = df["tags"].apply(literal_eval)
df["single_tag"] = df["tags"].apply(lambda x: x[0])
df.head()

,sentence_num,word,start_idx,end_idx,tags,single_tag,POS
0,0,This,0,4,[B-Temporal],B-Temporal,PRON
1,0,week,5,9,[I-Temporal],I-Temporal,NOUN
2,0,sees,10,14,[O],O,VERB
3,0,the,15,18,[O],O,PRON
4,0,start,19,24,[O],O,VERB


In [5]:
words = set(list(df['word'].values))
words.add('PADword')
n_words = len(words)
print(f"There are {n_words} unique words")

There are 4332 unique words


In [6]:
tags = list(set(df["single_tag"].values))
n_tags = len(tags)
print(f"There are {n_tags} unique tags")

There are 29 unique tags


In [7]:
class SentenceGetter:
    """Iterator to get a sentence sequence and its BIO tags"""
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, pos, t) for w, pos, t in zip(s["word"].values.tolist(), s["POS"].values.tolist(), s["single_tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence_num").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [8]:
getter = SentenceGetter(df)
sentences = getter.sentences
print(f"There are {len(sentences)} total sentence")

There are 1013 total sentence


In [9]:
# Make indices for ML modeling
words2index = {w:i for i,w in enumerate(words)}
tags2index = {t:i for i,t in enumerate(tags)}

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 50
y = [[tags2index[w[-1]] for w in s] for s in sentences]

y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tags2index["O"])

In [11]:
y.shape

(1013, 50)

In [12]:
max_len = 50
X = [[words2index[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=words2index["PADword"])

In [13]:
import tensorflow as tf

x_tensor = tf.convert_to_tensor(X)
y_tensor = tf.convert_to_tensor(y)

ds = tf.data.Dataset.from_tensor_slices((x_tensor, y_tensor))

BATCH_SIZE = 64

2022-11-15 15:32:56.016537: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-11-15 15:32:56.036834: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2022-11-15 15:32:56.036856: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: enyquist-X399-DESIGNARE-EX
2022-11-15 15:32:56.036861: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: enyquist-X399-DESIGNARE-EX
2022-11-15 15:32:56.036948: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.85.2
2022-11-15 15:32:56.036966: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 520.61.5
2022-11-15 15:32:56.036971: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 520.61.5 does not

In [14]:
def get_dataset_partitions_tf(
    ds, 
    ds_size, 
    train_split=0.8, 
    val_split=0.1, 
    test_split=0.1, 
    shuffle=True, 
    shuffle_size=1000
):
    assert (train_split + test_split + val_split) == 1

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=42)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds.batch(BATCH_SIZE), val_ds.batch(BATCH_SIZE), test_ds.batch(BATCH_SIZE)

In [15]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(
    ds=ds,
    ds_size=X.shape[0],
)

In [16]:
import io
from pathlib import Path
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, TimeDistributed, Input, Dropout
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'keras'

In [17]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow_addons.text import crf_log_likelihood, crf_decode

"""
Credit:
https://github.com/ngoquanghuy99/POS-Tagging-BiLSTM-CRF
"""

class CRF(L.Layer):
    def __init__(self,
                 output_dim,
                 sparse_target=True,
                 **kwargs):
        """    
        Args:
            output_dim (int): the number of labels to tag each temporal input.
            sparse_target (bool): whether the the ground-truth label represented in one-hot.
        Input shape:
            (batch_size, sentence length, output_dim)
        Output shape:
            (batch_size, sentence length, output_dim)
        """
        super(CRF, self).__init__(**kwargs)
        self.output_dim = int(output_dim) 
        self.sparse_target = sparse_target
        self.input_spec = L.InputSpec(min_ndim=3)
        self.supports_masking = False
        self.sequence_lengths = None
        self.transitions = None

    def build(self, input_shape):
        assert len(input_shape) == 3
        f_shape = tf.TensorShape(input_shape)
        input_spec = L.InputSpec(min_ndim=3, axes={-1: f_shape[-1]})

        if f_shape[-1] is None:
            raise ValueError('The last dimension of the inputs to `CRF` '
                             'should be defined. Found `None`.')
        if f_shape[-1] != self.output_dim:
            raise ValueError('The last dimension of the input shape must be equal to output'
                             ' shape. Use a linear layer if needed.')
        self.input_spec = input_spec
        self.transitions = self.add_weight(name='transitions',
                                           shape=[self.output_dim, self.output_dim],
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.built = True

    def compute_mask(self, inputs, mask=None):
        # Just pass the received mask from previous layer, to the next layer or
        # manipulate it if this layer changes the shape of the input
        return mask

    def call(self, inputs, sequence_lengths=None, training=None, **kwargs):
        sequences = tf.convert_to_tensor(inputs, dtype=self.dtype)
        if sequence_lengths is not None:
            assert len(sequence_lengths.shape) == 2
            assert tf.convert_to_tensor(sequence_lengths).dtype == 'int32'
            seq_len_shape = tf.convert_to_tensor(sequence_lengths).get_shape().as_list()
            assert seq_len_shape[1] == 1
            self.sequence_lengths = K.flatten(sequence_lengths)
        else:
            self.sequence_lengths = tf.ones(tf.shape(inputs)[0], dtype=tf.int32) * (
                tf.shape(inputs)[1]
            )

        viterbi_sequence, _ = crf_decode(sequences,
                                         self.transitions,
                                         self.sequence_lengths)
        output = K.one_hot(viterbi_sequence, self.output_dim)
        return K.in_train_phase(sequences, output)

    @property
    def loss(self):
        def crf_loss(y_true, y_pred):
            y_pred = tf.convert_to_tensor(y_pred, dtype=self.dtype)
            log_likelihood, self.transitions = crf_log_likelihood(
                y_pred,
                tf.cast(K.argmax(y_true), dtype=tf.int32) if self.sparse_target else y_true,
                self.sequence_lengths,
                transition_params=self.transitions,
            )
            return tf.reduce_mean(-log_likelihood)
        return crf_loss

    @property
    def accuracy(self):
        def viterbi_accuracy(y_true, y_pred):
            # -1e10 to avoid zero at sum(mask)
            mask = K.cast(
                K.all(K.greater(y_pred, -1e10), axis=2), K.floatx())
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            if self.sparse_target:
                y_true = K.argmax(y_true, 2)
            y_pred = K.cast(y_pred, 'int32')
            y_true = K.cast(y_true, 'int32')
            corrects = K.cast(K.equal(y_true, y_pred), K.floatx())
            return K.sum(corrects * mask) / K.sum(mask)
        return viterbi_accuracy

    @property
    def f1(self):
        def crf_f1(y_true, y_pred):
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
            possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
            predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
            precision = true_positives / (predicted_positives + K.epsilon())
            recall = true_positives / (possible_positives + K.epsilon())
            f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
            return f1_val
        return crf_f1

    def compute_output_shape(self, input_shape):
        tf.TensorShape(input_shape).assert_has_rank(3)
        return input_shape[:2] + (self.output_dim,)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'sparse_target': self.sparse_target,
            'supports_masking': self.supports_masking,
            'transitions': K.eval(self.transitions)
        }
        base_config = super(CRF, self).get_config()
        return dict(base_config, **config)


In [18]:
EMBEDDING_DIR = DATA_DIR / "embeddings"

In [19]:
# standard libaries
import io
from pathlib import Path
from typing import Dict, Tuple

# third party libraries
import numpy as np
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models


def create_model(
    vocab_size: int, max_length: int, embedding_dim: int, word_index: Dict[str, int], tag_index: Dict[str, int]
) -> Tuple[models.Model]:
    """Create Bi-LSTM CRF model in tensorflow.

    Model1 is the trainable model. Model2 is for predictions and returns:
    [predicted labels, LSTM hidden state (Forward and backward), LSTM cell state (forward and backward), embeddings]

    This is leveraged to build the REINFORCE states.

    Adapted from:
    https://github.com/ngoquanghuy99/POS-Tagging-BiLSTM-CRF

    Args:
        vocab_size (int): Size of vocabulary
        max_length (int): Max sequence length
        embedding_dim (int): Size of embedding. Make sure to match size of GloVe embedding.
        word_index (Dict[str, int]): Index mapping words to ints
        tag_index (Dict[str, int]): Index mapping tokens to ints

    Returns:
        Tuple[Model]: Compiled Model and Non-compiled Model
        with exposed LSTM and embedding layers
    """

    embeddings_index = {}
    with io.open(EMBEDDING_DIR / "glove.6B.100d.txt", "r", encoding="utf-8") as f:
        for line in f:
            values = line.strip().split()
            curr_word = values[0]
            coefs = np.asarray(values[1:], dtype="float64")
            embeddings_index[curr_word] = coefs
        embeddings_matrix = np.zeros((vocab_size, embedding_dim))
        for word, i in word_index.items():
            if i > vocab_size:
                continue
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embeddings_matrix[i] = embedding_vector

    inputs = layers.Input(shape=(max_length, ))

    embeddings = layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=max_length,
        weights=[embeddings_matrix],
        mask_zero=True
    )(inputs)

    lstm_out, sh_fw, sc_fw, sh_bw, sc_bw = layers.Bidirectional(
        layers.LSTM(
            units=embedding_dim, return_sequences=True, return_state=True, recurrent_dropout=0.01
        )
    )(embeddings)

    time_dist = layers.TimeDistributed(layers.Dense(len(tag_index)))(lstm_out)
    
    crf = CRF(len(tag_index), sparse_target=False)
    pred = crf(time_dist)

    model1 = models.Model(inputs=[inputs], outputs=[pred])
    model2 = models.Model(inputs=[inputs], outputs=[pred, sh_fw, sc_fw, sh_bw, sc_bw, embeddings])

    model1.compile(optimizer="adam", loss=crf.loss, metrics=[crf.accuracy])
    model1.summary()

    return model1, model2

In [20]:
import shutil

log_dir = ROOT_DIR / "models/logs/"

if any(log_dir.iterdir()):
    for i in log_dir.glob("**/*"):
        if i.is_dir():
            shutil.rmtree(i)
        else:
            i.unlink()

In [21]:
model1, model2 = create_model(
    vocab_size=len(words2index),
    max_length=50,
    embedding_dim=100,
    word_index=words2index,
    tag_index=tags2index
)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 100)           433200    
_________________________________________________________________
bidirectional (Bidirectional [(None, 50, 200), (None,  160800    
_________________________________________________________________
time_distributed (TimeDistri (None, 50, 29)            5829      
_________________________________________________________________
crf (CRF)                    (None, 50, 29)            841       
Total params: 600,670
Trainable params: 600,670
Non-trainable params: 0
_________________________________________________________________


In [22]:
callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3)
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, 
    histogram_freq=1,
    profile_batch="1,20"
)


history = model1.fit(
    train_ds,
    epochs=100, 
    verbose=1,
    validation_data=val_ds,
    callbacks=[callback, tensorboard_callback],
)

2022-11-15 15:33:12.847610: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2022-11-15 15:33:12.925520: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.


Epoch 1/100
13/13 [==============================] - 3s 195ms/step - loss: 114.6862 - viterbi_accuracy: 0.6133 - val_loss: 128.0542 - val_viterbi_accuracy: 0.8278
Epoch 2/100
 6/13 [============>.................] - ETA: 0s - loss: 51.8897 - viterbi_accuracy: 0.8465

2022-11-15 15:33:24.019861: I tensorflow/core/profiler/rpc/client/save_profile.cc:168] Creating directory: /home/enyquist/repos/RLNER/models/logs/train/plugins/profile/2022_11_15_15_33_22
2022-11-15 15:33:26.394796: I tensorflow/core/profiler/rpc/client/save_profile.cc:174] Dumped gzipped tool data for trace.json.gz to /home/enyquist/repos/RLNER/models/logs/train/plugins/profile/2022_11_15_15_33_22/enyquist-X399-DESIGNARE-EX.trace.json.gz
2022-11-15 15:33:27.533344: I tensorflow/core/profiler/utils/event_span.cc:288] Generation of step-events took 136.557 ms

2022-11-15 15:33:27.551659: I tensorflow/python/profiler/internal/profiler_wrapper.cc:87] Creating directory: /home/enyquist/repos/RLNER/models/logs/train/plugins/profile/2022_11_15_15_33_22Dumped tool data for overview_page.pb to /home/enyquist/repos/RLNER/models/logs/train/plugins/profile/2022_11_15_15_33_22/enyquist-X399-DESIGNARE-EX.overview_page.pb
Dumped tool data for input_pipeline.pb to /home/enyquist/repos/RLNER/models/lo

13/13 [==============================] - 9s 673ms/step - loss: 47.5260 - viterbi_accuracy: 0.8459 - val_loss: 126.5423 - val_viterbi_accuracy: 0.8430
Epoch 3/100
13/13 [==============================] - 1s 112ms/step - loss: 39.9703 - viterbi_accuracy: 0.8445 - val_loss: 125.8219 - val_viterbi_accuracy: 0.8486
Epoch 4/100
13/13 [==============================] - 1s 112ms/step - loss: 35.2214 - viterbi_accuracy: 0.8486 - val_loss: 126.0976 - val_viterbi_accuracy: 0.8470
Epoch 5/100
13/13 [==============================] - 1s 111ms/step - loss: 32.7713 - viterbi_accuracy: 0.8466 - val_loss: 126.1025 - val_viterbi_accuracy: 0.8347
Epoch 6/100
13/13 [==============================] - 1s 114ms/step - loss: 29.4232 - viterbi_accuracy: 0.8497 - val_loss: 125.8015 - val_viterbi_accuracy: 0.8417
Epoch 7/100
13/13 [==============================] - 1s 113ms/step - loss: 27.7148 - viterbi_accuracy: 0.8495 - val_loss: 124.2184 - val_viterbi_accuracy: 0.8690
Epoch 8/100
13/13 [=====================

The baseline Bi-LSTM CRF model has compiled, trained, and tested on the Re3d dataset successfully! Achieving ~99.00% Accuracy on the test set, which is pretty good, as only ~68% of the tags are "O" tags.

In [23]:
model1.evaluate(test_ds)

2/2 [==============================] - 0s 20ms/step - loss: 112.8591 - viterbi_accuracy: 0.9923


[112.85912322998047, 0.9923354983329773]

`create_model` creates two models, one which is trainable and one which is used for prediction. This model outputs:
* predictions
* LSTM hidden state forward pass
* LSTM cell state forward pass
* LSTM hidden state backward pass
* LSTM cell state backward pass
* Embeddings

This is leveraged in construction of the REINFORCE states.

In [24]:
preds = model2.predict(test_ds)

In [25]:
len(preds)

6

In [26]:
# The LSTM hidden state forward pass of the first test sequence
preds[1][0]

array([-0.9456301 , -0.20230967,  0.89551836,  0.30929264,  0.9070361 ,
       -0.7689723 ,  0.96961737, -0.7181937 , -0.96594965, -0.9800316 ,
        0.9646359 ,  0.09062003, -0.06115296, -0.46046507, -0.88109106,
        0.96693075, -0.88716763,  0.3860504 ,  0.5656328 , -0.99562603,
        0.83960235,  0.4403895 ,  0.997928  ,  0.9886104 , -0.91121924,
        0.19685347,  0.7100637 ,  0.8827886 ,  0.9306269 ,  0.95987576,
       -0.977393  , -0.895448  ,  0.89138913, -0.05796352,  0.7777271 ,
       -0.9391198 ,  0.26223004, -0.6093647 , -0.8600042 , -0.25359493,
        0.8589747 , -0.37789324,  0.9419344 , -0.82511884, -0.6723171 ,
        0.8910508 ,  0.862001  ,  0.9543241 ,  0.16341835,  0.48524523,
        0.8390463 ,  0.8130374 , -0.5667882 , -0.61088943,  0.91868997,
       -0.98205394,  0.9771145 , -0.9939071 , -0.9327623 , -0.9619404 ,
       -0.976128  , -0.29773775, -0.95627487,  0.09384784, -0.9693913 ,
        0.87953794,  0.83039594,  0.9909103 ,  0.9233074 ,  0.42

In [27]:
pred = model2.predict(tf.convert_to_tensor(X[0:1]))

In [28]:
pred[0][0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [29]:
X[0]

array([3033, 1946, 2120, 2021,  571, 3006, 4284, 3935,  542, 3006,    8,
        844, 2021, 2674, 3006, 2021,  350, 3587, 2313,  185, 3457, 3791,
       2021, 3878, 3006, 4065, 2709, 1635,  930,  921, 1069, 1025, 2149,
       4135, 2111,   24, 2944,  604,  522, 4011, 2021, 1825, 4141, 2634,
       1316, 1316, 1316, 1316, 1316, 1316], dtype=int32)

In [30]:
index2word = {idx: word for word, idx in words2index.items()}
index2tag = {idx: tag for tag, idx in tags2index.items()}

In [31]:
for pred, word_idx in zip(pred[0][0], X[0]):
    word = index2word[word_idx]
    tag_idx = np.argmax(pred)
    tag = index2tag[tag_idx]
    print(f"Word: {word} \tTag: {tag}")

Word: This 	Tag: B-Temporal
Word: week 	Tag: I-Temporal
Word: sees 	Tag: O
Word: the 	Tag: O
Word: start 	Tag: O
Word: of 	Tag: O
Word: a 	Tag: O
Word: second 	Tag: O
Word: round 	Tag: O
Word: of 	Tag: O
Word: talks 	Tag: O
Word: in 	Tag: O
Word: the 	Tag: O
Word: framework 	Tag: O
Word: of 	Tag: O
Word: the 	Tag: O
Word: European 	Tag: B-Organisation
Word: Union 	Tag: I-Organisation
Word: 's 	Tag: O
Word: Regional 	Tag: O
Word: Initiative 	Tag: O
Word: on 	Tag: O
Word: the 	Tag: O
Word: future 	Tag: O
Word: of 	Tag: O
Word: Syria 	Tag: B-Location
Word: , 	Tag: O
Word: High 	Tag: B-Person
Word: Representative 	Tag: I-Person
Word: Federica 	Tag: I-Person
Word: Mogherini 	Tag: I-Person
Word: ’s 	Tag: I-Person
Word: initiative 	Tag: O
Word: to 	Tag: O
Word: identify 	Tag: O
Word: post 	Tag: O
Word: - 	Tag: O
Word: conflict 	Tag: O
Word: arrangements 	Tag: O
Word: for 	Tag: O
Word: the 	Tag: B-Location
Word: country 	Tag: I-Location
Word: . 	Tag: O
Word: 
 	Tag: O
Word: PADword 	Tag: O
Wor

In [32]:
sentences[0]

[('This', 'PRON', 'B-Temporal'),
 ('week', 'NOUN', 'I-Temporal'),
 ('sees', 'VERB', 'O'),
 ('the', 'PRON', 'O'),
 ('start', 'VERB', 'O'),
 ('of', 'ADP', 'O'),
 ('a', 'PRON', 'O'),
 ('second', 'ADJ', 'O'),
 ('round', 'ADJ', 'O'),
 ('of', 'ADP', 'O'),
 ('talks', 'NOUN', 'O'),
 ('in', 'ADP', 'O'),
 ('the', 'PRON', 'O'),
 ('framework', 'NOUN', 'O'),
 ('of', 'ADP', 'O'),
 ('the', 'PRON', 'O'),
 ('European', 'ADJ', 'B-Organisation'),
 ('Union', 'NOUN', 'I-Organisation'),
 ("'s", 'AUX', 'O'),
 ('Regional', 'ADJ', 'O'),
 ('Initiative', 'NOUN', 'O'),
 ('on', 'ADP', 'O'),
 ('the', 'PRON', 'O'),
 ('future', 'NOUN', 'O'),
 ('of', 'ADP', 'O'),
 ('Syria', 'PROPN', 'B-Location'),
 (',', 'PUNCT', 'O'),
 ('High', 'ADJ', 'B-Person'),
 ('Representative', 'ADJ', 'I-Person'),
 ('Federica', 'PROPN', 'I-Person'),
 ('Mogherini', 'PROPN', 'I-Person'),
 ('’s', 'VERB', 'I-Person'),
 ('initiative', 'NOUN', 'O'),
 ('to', 'PART', 'O'),
 ('identify', 'VERB', 'O'),
 ('post', 'ADV', 'O'),
 ('-', 'PUNCT', 'O'),
 ('conf